# 02_markov_modeling_and_entropy

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import jensenshannon
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

DATA_PATH = r"c:/Users/asus/OneDrive/Desktop/ITML Project/processed_data/symbolized_data.csv"
OUTPUT_DIR = r"c:/Users/asus/OneDrive/Desktop/ITML Project/results"
PLOTS_DIR = r"c:/Users/asus/OneDrive/Desktop/ITML Project/plots"
TRANSITION_MATRICES_DIR = r"c:/Users/asus/OneDrive/Desktop/ITML Project/transition_matrices"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(TRANSITION_MATRICES_DIR, exist_ok=True)

def compute_shannon_entropy(probs):
    probs = probs[probs > 0]
    return -np.sum(probs * np.log2(probs))

def compute_transition_matrix(sequence, n_states=8):
    transitions = np.zeros((n_states, n_states))
    for (i, j) in zip(sequence[:-1], sequence[1:]):
        transitions[i, j] += 1
        
    # Laplace smoothing
    transitions += 1e-5
    row_sums = transitions.sum(axis=1)
    return transitions / row_sums[:, np.newaxis]

def compute_stationary_distribution(P):
    eigvals, eigvecs = np.linalg.eig(P.T)
    idx = np.argmin(np.abs(eigvals - 1))
    pi = np.real(eigvecs[:, idx])
    return pi / pi.sum()

def compute_entropy_rate(P, pi):
    H_rate = 0
    for i in range(len(pi)):
        row_entropy = 0
        for j in range(len(P[i])):
            if P[i, j] > 0:
                row_entropy -= P[i, j] * np.log2(P[i, j])
        H_rate += pi[i] * row_entropy
    return H_rate

def compute_kl_divergence(P_a, P_b):
    epsilon = 1e-10
    P_a = P_a + epsilon
    P_b = P_b + epsilon
    P_a /= P_a.sum()
    P_b /= P_b.sum()
    return np.sum(P_a * np.log2(P_a / P_b))

def compute_js_divergence_matrices(P_a, P_b):
    flat_a = P_a.flatten()
    flat_b = P_b.flatten()
    return jensenshannon(flat_a, flat_b, base=2) ** 2

def main():
    print("Loading symbolized data...")
    df = pd.read_csv(DATA_PATH)
    classes = df['device_class'].unique()
    n_states = 8
    
    results = []
    transition_matrices = {}
    stationary_distributions = {}
    
    print(f"Analyzing {len(classes)} classes...")
    
    for cls in classes:
        sequence = df[df['device_class'] == cls]['symbol'].values
        
        # 1. Markov Chain & Stationary Distribution
        P = compute_transition_matrix(sequence, n_states)
        pi = compute_stationary_distribution(P)
        
        transition_matrices[cls] = P
        stationary_distributions[cls] = pi
        
        # 2. Entropy Analysis
        h_rate = compute_entropy_rate(P, pi)
        h_marginal = compute_shannon_entropy(pi)
        
        results.append({
            'Class': cls,
            'Entropy Rate': h_rate,
            'Marginal Entropy': h_marginal
        })
        
        # Visualize Transition Matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(P, annot=True, fmt='.2f', cmap='Blues')
        plt.title(f'Transition Matrix - {cls}')
        plt.xlabel('To State')
        plt.ylabel('From State')
        plt.savefig(os.path.join(TRANSITION_MATRICES_DIR, f'transition_matrix_{cls}.png'))
        plt.close()
        
    results_df = pd.DataFrame(results)
    print("\nEntropy Analysis Results:\n", results_df)
    results_df.to_csv(os.path.join(OUTPUT_DIR, 'entropy_results.csv'), index=False)
    
    # Plot Entropy Rates
    plt.figure(figsize=(12, 6))
    sns.barplot(data=results_df, x='Class', y='Entropy Rate', palette='viridis')
    plt.xticks(rotation=45)
    plt.title('Entropy Rate by Drone Class')
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, 'entropy_rate_comparison.png'))
    plt.close()
    
    # --- Pairwise Divergence Analysis ---
    print("\nComputing Pairwise Divergences...")
    n_classes = len(classes)
    jsd_matrix = np.zeros((n_classes, n_classes))
    kl_matrix = np.zeros((n_classes, n_classes))
    
    for i in range(n_classes):
        for j in range(n_classes):
            cls_i, cls_j = classes[i], classes[j]
            jsd_matrix[i, j] = compute_js_divergence_matrices(transition_matrices[cls_i], transition_matrices[cls_j])
            kl_matrix[i, j] = compute_kl_divergence(stationary_distributions[cls_i], stationary_distributions[cls_j])
            
    # JSD Heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(jsd_matrix, xticklabels=classes, yticklabels=classes, cmap='coolwarm')
    plt.title('Jensen-Shannon Divergence (Transition Matrices)')
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, 'jsd_heatmap.png'))
    plt.close()
    
    # Dendrogram
    plt.figure(figsize=(10, 6))
    condensed_jsd = squareform(jsd_matrix, checks=False)
    Z = linkage(condensed_jsd, method='ward')
    dendrogram(Z, labels=classes, leaf_rotation=90)
    plt.title('Drone Clustering based on JSD of Markov Models')
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, 'dendrogram.png'))
    plt.close()
    
    print("Analysis complete.")

main()